In [17]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [21]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [23]:
train_dataset = datasets.ImageFolder(
    root=r"C:\Users\keval\OneDrive\Desktop\final_dataset\train",
    transform=train_transform
)

test_dataset = datasets.ImageFolder(
    root=r"C:\Users\keval\OneDrive\Desktop\final_dataset\test",
    transform=test_transform
)

print(train_dataset.class_to_idx)


{'ai': 0, 'real': 1}


In [25]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [27]:
model = models.resnet18(pretrained=True)


C:\Users\keval\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\keval\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\keval/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:12<00:00, 3.79MB/s]


In [29]:
for param in model.parameters():
    param.requires_grad = False


In [39]:
model.fc = nn.Linear(512, 2)
model = model.to(device)


In [43]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [45]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")


Epoch [1/5] Loss: 76.2060 | Accuracy: 82.08%
Epoch [2/5] Loss: 49.6756 | Accuracy: 89.62%
Epoch [3/5] Loss: 44.6745 | Accuracy: 90.68%
Epoch [4/5] Loss: 41.6361 | Accuracy: 91.20%
Epoch [5/5] Loss: 39.7580 | Accuracy: 91.42%


In [47]:

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 91.40%


In [49]:
torch.save(model.state_dict(), "resnet18_ai_vs_real.pth")
print("Model saved ✅")


Model saved ✅


In [51]:
infer_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [53]:
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = infer_transform(image).unsqueeze(0)  # (1, 3, 224, 224)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, 1)

    class_names = ["AI Generated", "Real Image"]

    print("Prediction:", class_names[predicted.item()])
    print("Confidence:", f"{confidence.item() * 100:.2f}%")


In [83]:
from PIL import Image

predict_image(
    r"C:\Users\keval\Downloads\Gemini_Generated_Image_aobdh7aobdh7aobd.png")


Prediction: Real Image
Confidence: 95.66%
